In [9]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 23.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy

In [12]:
#------------------------------
#import file with google colab from google drive
#------------------------------
from google.colab import drive
drive.mount("/content/drive")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
df_train = pd.read_csv("/content/drive/MyDrive/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/test.csv")
df_test_label = pd.read_csv("/content/drive/MyDrive/test_labels.csv")
df_sample_submission = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

In [5]:
#------------------------------
#import file local
#------------------------------

df_train = pd.read_csv("./dataset/train.csv")
df_test = pd.read_csv("./dataset/test.csv")
df_test_label = pd.read_csv("./dataset/test_labels.csv")
df_sample_submission = pd.read_csv("./dataset/sample_submission.csv")



FileNotFoundError: [Errno 2] No such file or directory: './dataset/train.csv'

In [14]:
#relier test et test_label
df_test = pd.merge(df_test, df_test_label, on='id')

#équilibrage des classes toxic, non toxic
print("Train before: ", df_train.shape)
print("Test before: ", df_test.shape)

#sous échantillonage df_train pour avoir 30000 lignes non toxiques
df_train_non_toxic = df_train[df_train['toxic'] == 0].sample(30000)
df_train_toxic = df_train[df_train['toxic'] == 1]
df_train = pd.concat([df_train_non_toxic, df_train_toxic])

#supprimer les lignes de df_test comprenant des -1 dans la partie toxic
df_test = df_test[df_test['toxic'] != -1]

#sous échantillonage de df_test pour avoir 20000 lignes non toxique
df_test_non_toxic = df_test[df_test['toxic'] == 0].sample(20000)
df_test_toxic = df_test[df_test['toxic'] == 1]
df_test = pd.concat([df_test_non_toxic, df_test_toxic])


#afficher le nombre de lignes et de colonnes
print("Train after: ", df_train.shape)
print("Test after: ", df_test.shape)



Train before:  (159571, 8)
Test before:  (153164, 8)
Train after:  (45294, 8)
Test after:  (26090, 8)


In [15]:
#preprocess

#supprimer les lignes avec des valeurs manquantes
df_train = df_train.dropna()
df_test = df_test.dropna()

#spacy preprocessing
nlp = spacy.load('en_core_web_md', disable=["parser", "ner"])

def preprocess_text(text):
  doc = nlp(text)
  tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
  return tokens

df_train['comment_text_tokenized'] = df_train['comment_text'].apply(preprocess_text)

df_test['comment_text_tokenized'] = df_test['comment_text'].apply(preprocess_text)

In [16]:
#save the preprocessed data into csv files
df_train.to_csv("train_preprocessed.csv", index=False)
df_test.to_csv("test_preprocessed.csv", index=False)